### Dependecies

In [1]:
!pip install chart_studio

     |████████████████████████████████| 64 kB 1.9 MB/s 


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import chart_studio.plotly as py
import plotly.express as px

### Loading the Data

In [28]:
covid = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv')

countries = covid.location.unique()
indexer = covid[covid.location == 'Ireland'].index
countries = covid.loc[indexer, 'date': 'new_deaths_per_million']
countries = countries.dropna()
countries.set_index('date', inplace=True)
countries.drop(countries.columns.difference(['new_deaths_per_million']), axis=1, inplace=True)
countries['MA30'] = countries.rolling(window=30).mean()['new_deaths_per_million']

In [29]:
fig = px.line(countries, countries.index, 'new_deaths_per_million', labels={'x':'Date'})
fig.add_scatter(x=countries.index, y=countries.MA30)